In [1]:
import scipy.fft as fft
from scipy.stats import skew, kurtosis
from utils import _to_float, load_data
import itertools
import math
import numpy as np
import skimage.io as io
from skimage.color import rgb2gray
import matplotlib.pyplot as plt

In [2]:
file_path_img_r = '../../samples/Catec_Two_PlateIQI_20um/Catec_Two_PlateIQI_20um_1620proj_220kV_Rayscan-SimCT_800x800x1000_16bit.raw'
file_path_img_m = '../../samples/Catec_Two_PlateIQI_20um/Catec_Two_PlateIQI_20um_810proj_220kV_Rayscan-SimCT_800x800x1000_16bit.raw'
# # Data loading
# img_r = load_data(file_path_img_r, data_range=255, normalize=True, batch=False)[400:600, 300, 300:500]
# img_m = load_data(file_path_img_m, data_range=255, normalize=True, batch=False)[400:600, 300, 300:500]

img_r = load_data(file_path_img_r, data_range=255, normalize=True, batch=False)[0:1000, 200, 0:800]
img_m = load_data(file_path_img_m, data_range=255, normalize=True, batch=False)[0:1000, 200, 0:800]

# file_path_img_r = '../../TID2013/reference_images/I01.bmp'
# file_path_img_m = '../../TID2013/distorted_images/I01_01_2.bmp'
# 
# img_r = io.imread(file_path_img_r)
# img_m = io.imread(file_path_img_m)
# 
# img_r = rgb2gray(img_r)
# img_m = rgb2gray(img_m)

In [3]:
M, N = img_r.shape
account_monitor = False

In [4]:
# accounting for display resolution and viewing distance
if account_monitor:
    disp_res = 81.59  # px / in
    view_dis = 39.37  # in
    cycles_per_degree = (disp_res * view_dis * np.tan(np.pi / 180)) / 2
else:
    cycles_per_degree = 32

In [5]:
def _contrast_sensitivity_function(m, n, nfreq):
    xplane, yplane = np.meshgrid(np.arange(-n / 2 + 0.5, n / 2 + 0.5), np.arange(-m / 2 + 0.5, m / 2 + 0.5))
    plane = (xplane + 1j * yplane) * 2 * nfreq / n
    radfreq = np.abs(plane)
    
    w = 0.7
    theta = np.angle(plane)
    s = ((1 - w) / 2) * np.cos(4 * theta) + ((1 + w) / 2)
    radfreq /= s
    
    lambda_ = 0.114
    f_peak = 7.8909
    cond = radfreq < f_peak
    csf = 2.6 * (0.0192 + lambda_ * radfreq) * np.exp(-(lambda_ * radfreq)**1.1)
    csf[cond] = 0.9809
    
    return csf

In [6]:
csf = _contrast_sensitivity_function(M, N, 32)

In [8]:
k = 0.02874
gamma = 2.2
LUT = np.arange(0, 256)
LUT = k * LUT ** (gamma / 3)
lum_r = LUT[img_r]
lum_m = LUT[img_m]

In [9]:
def _fft(img):
    return fft.fftshift(fft.fftn(img))

def _ifft(fourier_img):
    return fft.ifftn(fft.ifftshift(fourier_img))

In [10]:
lum_r_fft = _fft(lum_r)
lum_m_fft = _fft(lum_m)

I_org = np.real(_ifft(csf * lum_r_fft))
I_dst = np.real(_ifft(csf * lum_m_fft))

I_err = I_dst - I_org

In [11]:
def extract_blocks(img, block_size, stride):
    """
    Extracts blocks from an image.
    :param img: Input image
    :param block_size: Size of the block
    :param stride: Stride
    :return: Numpy array of blocks
    """
    boxes = []
    m, n = img.shape
    for i in range(0, m - (block_size - 1), stride):
        for j in range(0, n - (block_size - 1), stride):
            boxes.append(img[i:i + block_size, j:j + block_size])
    return np.array(boxes)

In [42]:
def min_std(image):
    tmp = np.empty(image.shape)
    stdout = np.empty(image.shape)
    for i in range(0, M-15, 4):
        for j in range(0, N-15, 4):
            mean = 0
            for k in range(i, i+8):
                for l in range(j, j+8):
                    mean += image[k, l]
            mean /= 64
            
            stdev = 0
            for k in range(i, i+8):
                for l in range(j, j+8):
                    stdev += (image[k, l] - mean) ** 2
            stdev = np.sqrt(stdev / 63)
            
            for k in range(i, i+4):
                for l in range(j, j+4):
                    tmp[k, l] = stdev
                    stdout[k, l] = stdev
    
    for i in range(0, M-15, 4):
        for j in range(0, N-15, 4):
            val = tmp[i, j]
            for k in range(i, i+8, 5):
                for l in range(j, j+8, 5):
                    if tmp[k, l] < val:
                        val = tmp[k, l]
                        
            for k in range(i, i+4):
                for l in range(j, j+4):
                    stdout[k, l] = val
    
    return stdout            

In [13]:
# Contrast masking
BLOCK_SIZE = 16
overlap = 0.75
STRIDE = BLOCK_SIZE - int(overlap * BLOCK_SIZE)
I_org_blocks = extract_blocks(I_org, block_size=BLOCK_SIZE, stride=STRIDE)
I_err_blocks = extract_blocks(I_err, block_size=BLOCK_SIZE, stride=STRIDE)

In [46]:
mu_org_p = np.mean(I_org_blocks, axis=(1, 2))
# std_org_p = np.array([min_std(block, block_size=int(BLOCK_SIZE / 2), stride=int(BLOCK_SIZE / 2)) for block in I_org_blocks])
std_org = min_std(I_org)
std_err_p = np.std(I_err_blocks, axis=(1, 2), ddof=1)

mu_org = np.zeros(I_org.shape)
# std_org = np.zeros(I_org.shape)
std_err = np.zeros(I_err.shape)

block_n = 0
for x in range(0, I_org.shape[0] - STRIDE * 3, STRIDE):
    for y in range(0, I_org.shape[1] - STRIDE * 3, STRIDE):
        mu_org[x:x+STRIDE, y:y+STRIDE] = mu_org_p[block_n]
        # std_org[x:x+STRIDE, y:y+STRIDE] = std_org_p[block_n]
        std_err[x:x+STRIDE, y:y+STRIDE] = std_err_p[block_n]
        block_n += 1
del mu_org_p, std_org_p, std_err_p

In [44]:
C_org = std_org / mu_org
C_err = np.zeros(std_err.shape)
np.divide(std_err, mu_org, out=C_err, where=mu_org>0.5)

C:\Users\p42938\AppData\Local\Temp\ipykernel_7788\863585825.py:1: RuntimeWarning: invalid value encountered in divide
  C_org = std_org / mu_org


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [47]:
Ci_org = np.log(C_org)
Ci_err = np.log(C_err)

C:\Users\p42938\AppData\Local\Temp\ipykernel_7788\2523429962.py:1: RuntimeWarning: divide by zero encountered in log
  Ci_org = np.log(C_org)
C:\Users\p42938\AppData\Local\Temp\ipykernel_7788\2523429962.py:2: RuntimeWarning: divide by zero encountered in log
  Ci_err = np.log(C_err)


In [45]:
delta = -5
cond_1 = np.logical_and(Ci_err > Ci_org, Ci_org > delta)
cond_2 = np.logical_and(Ci_err > delta, delta >= Ci_org)

xi = np.zeros(C_err.shape)
np.subtract(np.log(C_err), np.log(C_org), out=xi, where=cond_1)
np.subtract(np.log(C_err), delta, out=xi, where=cond_2)

C:\Users\p42938\AppData\Local\Temp\ipykernel_7788\913300767.py:2: RuntimeWarning: divide by zero encountered in log
  cond_1 = np.logical_and(np.log(C_err) > np.log(C_org), np.log(C_org) > delta)
C:\Users\p42938\AppData\Local\Temp\ipykernel_7788\913300767.py:3: RuntimeWarning: divide by zero encountered in log
  cond_2 = np.logical_and(np.log(C_err) > delta, delta >= np.log(C_org))
C:\Users\p42938\AppData\Local\Temp\ipykernel_7788\913300767.py:6: RuntimeWarning: divide by zero encountered in log
  np.subtract(np.log(C_err), np.log(C_org), out=xi, where=cond_1)
C:\Users\p42938\AppData\Local\Temp\ipykernel_7788\913300767.py:7: RuntimeWarning: divide by zero encountered in log
  np.subtract(np.log(C_err), delta, out=xi, where=cond_2)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
I_err.shape

(1000, 800)

In [33]:
# Combination of contrast masking and contrast sensitivity
D_p = np.sum(I_err ** 2) / (I_err.shape[0] * I_err.shape[1]) 

In [34]:
d_detect = np.sqrt(np.sum((xi * D_p) ** 2) / len(xi))
d_detect

0.02028448380905756

In [17]:
def _gabor_convolve(im, scales_num: int, orientations_num: int, min_wavelength=3, wavelength_scaling=3, bandwidth_param=0.55, d_theta_on_sigma=1.5):
    """
    Computes Gabor filter responses. \n
    bandwidth_param vs wavelength_scaling \n
    0.85 <--> 1.3 \n
    0.74 <--> 1.6 (1 octave bandwidth) \n
    0.65 <--> 2.1 \n
    0.55 <--> 3.0 (2 octave bandwidth) \n
    
    AUTHOR
    ------
    This code was originally written in Matlab by Peter Kovesi and adapted by Eric Larson. \n
    It is available from http://vision.eng.shizuoka.ac.jp/mad (version 2011_10_07). \n
    It was translated to Python by Lukas Behammer. \n
    
    Author: Peter Kovesi \n
    Department of Computer Science & Software Engineering \n
    The University of Western Australia \n
    pk@cs.uwa.edu.au  https://peterkovesi.com/projects/ \n
    
    Adaption: Eric Larson \n
    Department of Electrical and Computer Engineering \n
    Oklahoma State University, 2008 \n
    University Of Washington Seattle, 2009 \n
    Image Coding and Analysis lab
    
    Translation: Lukas Behammer \n
    Research Center Wels \n
    University of Applied Sciences Upper Austria \n
    CT Research Group \n
    
    MODIFICATIONS
    -------------
    May 2001 \n
    Altered, 2008, Eric Larson \n
    Altered precomputations, 2011, Eric Larson \n
    Translated to Python, 2024, Lukas Behammer
    
    Literature
    -------
    D. J. Field, "Relations Between the Statistics of Natural Images and the
    Response Properties of Cortical Cells", Journal of The Optical Society of
    America A, Vol 4, No. 12, December 1987. pp 2379-2394
    
    LICENSE
    -------
    Copyright (c) 2001-2010 Peter Kovesi
    www.peterkovesi.com

    Permission is hereby granted, free of charge, to any person obtaining a copy
    of this software and associated documentation files (the "Software"), to deal
    in the Software without restriction, subject to the following conditions:

    The above copyright notice and this permission notice shall be included in 
    all copies or substantial portions of the Software.

    The Software is provided "as is", without warranty of any kind.
    :param im: Image to be filtered
    :param scales_num: Number of wavelet scales
    :param orientations_num: Number of filter orientations
    :param min_wavelength: Wavelength of smallest scale filter
    :param wavelength_scaling: Scaling factor between successive filters
    :param bandwidth_param: Ratio of standard deviation of the Gaussian describing log Gabor filter's transfer function in the frequency domain to the filter's center frequency (0.74 for 1 octave bandwidth, 0.55 for 2 octave bandwidth, 0.41 for 3 octave bandwidth)
    :param d_theta_on_sigma: Ratio of angular interval between filter orientations and standard deviation of angular Gaussian function used to construct filters in the frequency plane
    :return: 
    """
    # Precomputing and assigning variables
    scales = np.arange(0, scales_num)
    orientations = np.arange(0, orientations_num)
    rows, cols = im.shape # image dimensions
    # center of image
    col_c = math.floor(cols/2)
    row_c = math.floor(rows/2)
    
    # set up filter wavelengths from scales
    wavelengths = [min_wavelength * wavelength_scaling ** scale_n for scale_n in range(0, scales_num)]
    
    # convert image to frequency domain
    im_fft = fft.fftn(im)
    
    # compute matrices of same site as im with values ranging from -0.5 to 0.5 (-1.0 to 1.0) for horizontal and vertical directions each
    if cols % 2 == 0:
        x_range = np.linspace(-cols/2, (cols-2)/2, cols) / (cols / 2)
    else:
        x_range = np.linspace(-cols/2, cols/2, cols) / (cols / 2)
    if rows % 2 == 0:
        y_range = np.linspace(-rows/2, (rows-2)/2, rows) / (rows / 2)
    else:
        y_range = np.linspace(-rows/2, rows/2, rows) / (rows / 2)
    x, y = np.meshgrid(x_range, y_range)
    
    # filters have radial component (frequency band) and an angular component (orientation), those are multiplied to get the final filter
    
    # compute radial distance from center of matrix
    radius = np.sqrt(x**2 + y**2)
    radius[radius == 0] = 1  # avoid logarithm of zero
    
    # compute polar angle and its sine and cosine
    theta = np.arctan2(-y, x)
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    
    # compute standard deviation of angular Gaussian function
    theta_sigma = np.pi / orientations_num / d_theta_on_sigma
    
    # compute radial component
    radial_components = []
    for scale_n, scale in enumerate(scales):  # for each scale
        center_freq = 1.0 / wavelengths[scale_n]  # center frequency of filter
        normalised_center_freq = center_freq / 0.5
        # log Gabor response for each frequency band (scale)
        log_gabor = np.exp((np.log(radius) - np.log(normalised_center_freq))**2 / -(2 * np.log(bandwidth_param) ** 2))
        log_gabor[row_c, col_c] = 0
        radial_components.append(log_gabor)
    
    # angular component and final filtering    
    res = np.empty((scales_num, orientations_num), dtype=object)  # precompute result array
    for orientation_n, orientation in enumerate(orientations):  # for each orientation
        # compute angular component
        # Pre-compute filter data specific to this orientation
        # For each point in the filter matrix calculate the angular distance from the specified filter orientation.  To overcome the angular wrap-around problem sine difference and cosine difference values are first computed and then the atan2 function is used to determine angular distance.
        angle = orientation_n * np.pi / orientations_num  # filter angle
        diff_sin = sin_theta * np.cos(angle) - cos_theta * np.sin(angle)  # difference of sin
        diff_cos = cos_theta * np.cos(angle) + sin_theta * np.sin(angle)  # difference of cos
        angular_distance = abs(np.arctan2(diff_sin, diff_cos))  # absolute angular distance
        spread = np.exp((-angular_distance ** 2) / (2 * theta_sigma ** 2))  # angular filter component
        
        # filtering
        for scale_n, scale in enumerate(scales):  # for each scale
            # compute final filter
            filter_ = fft.fftshift(radial_components[scale_n] * spread)
            filter_[0, 0] = 0
            
            # apply filter
            res[scale_n, orientation_n] = fft.ifftn(im_fft * filter_)
    
    return res

In [18]:
gabor_org = _gabor_convolve(img_r, nscale=5, norient=4)
gabor_dst = _gabor_convolve(img_m, nscale=5, norient=4)

In [19]:
def _get_statistics(image, block_size, stride):
    sub_blocks = extract_blocks(image, block_size=block_size, stride=stride)
    std = np.std(sub_blocks, axis=(1, 2))
    sk = []
    k = []
    for block in sub_blocks:
        sk.append(skew(np.abs(block.flatten())))
        k.append(kurtosis(np.abs(block.flatten())))
    return std, sk, k

In [20]:
stds_r = []
sks_r = []
ks_r = []
for s_o in gabor_org.reshape(20):
    std, sk, k = _get_statistics(s_o, block_size=BLOCK_SIZE, stride=STRIDE)
    stds_r.append(std)
    sks_r.append(sk)
    ks_r.append(k)
    
stds_m = []
sks_m = []
ks_m = []
for s_o in gabor_dst.reshape(20):
    std, sk, k = _get_statistics(s_o, block_size=BLOCK_SIZE, stride=STRIDE)
    stds_m.append(std)
    sks_m.append(sk)
    ks_m.append(k)

In [21]:
stds_r = np.array(stds_r)
sks_r = np.array(sks_r)
ks_r = np.array(ks_r)
stds_m = np.array(stds_m)
sks_m = np.array(sks_m)
ks_m = np.array(ks_m)

In [22]:
stats = np.abs(stds_r - stds_m) + 2 * np.abs(sks_r - sks_m) + np.abs(ks_r - ks_m)

In [23]:
w = [0.5, 0.75, 1, 5, 6]
ws = np.array(list((itertools.repeat(w, 4)))).flatten()

In [24]:
ws = np.array(list(itertools.repeat(ws, stats.shape[1]))).T

In [25]:
eta = np.sum(ws * stats, axis=0)

In [26]:
d_appear = np.sqrt(np.sum(eta ** 2) / len(eta))
d_appear

64.681246703985

In [27]:
beta_1 = 0.467
beta_2 = 0.130
alpha = 1 / (1 + beta_1 * d_detect**beta_2)
mad_index = d_detect**alpha / d_appear**(1-alpha)

In [28]:
mad_index

0.011170525941570184